Michael D'Acampora
Data612 - Project 1
Global Baseline Predictors and RMSE
Summer 2019

-----------

TODO: DESCRIBE THE RECOMMENDER SYSTEM FROM A BUSINESS CASE
The recommender system business case is to recommend a genre of music 






In [102]:
import numpy as np
import pandas as pd
from random import randint

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

After loading our imports we create an 10x8 dataframe with random scores from 0 to 5.

In [103]:
users = ['user1', 'user2', 'user3', 'user4', 'user5', 'user6', 'user7', 'user8', 'user9', 'user10']
ratings = {
    "Users": [i for i in users],
    "Rock": [randint(1,5) for i in range(10)],
    "Jazz": [randint(1,5) for i in range(10)],
    "Pop": [randint(1,5) for i in range(10)],
    "HipHop": [randint(1,5) for i in range(10)],
    "Classical": [randint(1,5) for i in range(10)],
    "Blues": [randint(1,5) for i in range(10)],
    "Country": [randint(1,5) for i in range(10)],
    "Folk": [randint(1,5) for i in range(10)],
}

The output of the dataframe is shown below.

In [104]:
df = pd.DataFrame(ratings).set_index('Users')
df

,Rock,Jazz,Pop,HipHop,Classical,Blues,Country,Folk
Users,,,,,,,,
user1,1,1,4,1,4,5,1,3
user2,2,1,2,1,5,1,4,5
user3,3,2,1,2,5,5,2,4
user4,4,5,2,3,4,4,5,5
user5,5,2,4,5,2,1,3,5
user6,5,3,5,4,4,2,4,1
user7,5,2,1,5,2,4,1,1
user8,1,5,5,3,2,5,2,1
user9,2,2,5,2,3,1,2,4


In order to add null values to represent a more realistic scenario, we choose at randome about 15% to be NaN and output the revised dataframe below.

In [105]:
np.random.seed(111)
mask = np.random.choice([True, False], size=df.shape, p=[0.15, 0.85])
df = df.mask(mask)    

In [106]:
df

,Rock,Jazz,Pop,HipHop,Classical,Blues,Country,Folk
Users,,,,,,,,
user1,1.0,1.0,4.0,1.0,4.0,NaN,NaN,3.0
user2,2.0,1.0,2.0,1.0,NaN,1.0,4.0,5.0
user3,3.0,NaN,NaN,2.0,5.0,5.0,2.0,4.0
user4,4.0,5.0,2.0,NaN,4.0,NaN,5.0,5.0
user5,5.0,2.0,4.0,NaN,2.0,1.0,3.0,5.0
user6,5.0,3.0,5.0,4.0,4.0,2.0,NaN,1.0
user7,5.0,2.0,1.0,5.0,2.0,4.0,NaN,NaN
user8,NaN,NaN,NaN,3.0,2.0,5.0,2.0,NaN
user9,2.0,2.0,5.0,2.0,3.0,1.0,2.0,4.0


From here the dataframe is split into training and test sets on an 80/20 basis.

In [107]:
train, test = train_test_split(df, test_size=0.2)

The next step is to calculate the raw averages of the training and test sets. First we will work on the training set.

In [108]:
train

,Rock,Jazz,Pop,HipHop,Classical,Blues,Country,Folk
Users,,,,,,,,
user1,1.0,1.0,4.0,1.0,4.0,NaN,NaN,3.0
user8,NaN,NaN,NaN,3.0,2.0,5.0,2.0,NaN
user5,5.0,2.0,4.0,NaN,2.0,1.0,3.0,5.0
user2,2.0,1.0,2.0,1.0,NaN,1.0,4.0,5.0
user4,4.0,5.0,2.0,NaN,4.0,NaN,5.0,5.0
user3,3.0,NaN,NaN,2.0,5.0,5.0,2.0,4.0
user6,5.0,3.0,5.0,4.0,4.0,2.0,NaN,1.0
user7,5.0,2.0,1.0,5.0,2.0,4.0,NaN,NaN


In [109]:
# sum the rows, them sum the row sums
# then divide the total by the non-NaN count to obtain the raw avg
train_row_sum = train.sum(axis=1)
df_train_sum = train_row_sum.sum()
train_raw_avg = df_train_sum / train.count().sum()
print(f'The raw average of the training set is: {"%.4f" % train_raw_avg}')

The raw average of the training set is: 3.1224


Next we will impute (replace) any NaN values in the training set with the raw average with the output below.

In [110]:
train_imputed = train.fillna(train_raw_avg)
train_imputed

,Rock,Jazz,Pop,HipHop,Classical,Blues,Country,Folk
Users,,,,,,,,
user1,1.000000,1.000000,4.000000,1.000000,4.000000,3.122449,3.122449,3.000000
user8,3.122449,3.122449,3.122449,3.000000,2.000000,5.000000,2.000000,3.122449
user5,5.000000,2.000000,4.000000,3.122449,2.000000,1.000000,3.000000,5.000000
user2,2.000000,1.000000,2.000000,1.000000,3.122449,1.000000,4.000000,5.000000
user4,4.000000,5.000000,2.000000,3.122449,4.000000,3.122449,5.000000,5.000000
user3,3.000000,3.122449,3.122449,2.000000,5.000000,5.000000,2.000000,4.000000
user6,5.000000,3.000000,5.000000,4.000000,4.000000,2.000000,3.122449,1.000000
user7,5.000000,2.000000,1.000000,5.000000,2.000000,4.000000,3.122449,3.122449


Lastly an equivalent sized dataframe is created full of raw averages so we can calculate the root mean square error (RMSE).

In [111]:
train_avg = {
    "col1": [train_raw_avg for i in range(8)],
    "col2": [train_raw_avg for i in range(8)],
    "col3": [train_raw_avg for i in range(8)],
    "col4": [train_raw_avg for i in range(8)],
    "col5": [train_raw_avg for i in range(8)],
    "col6": [train_raw_avg for i in range(8)],
    "col7": [train_raw_avg for i in range(8)],
    "col8": [train_raw_avg for i in range(8)],
}

train_raw_avg_df = pd.DataFrame(train_avg)
train_raw_avg_df

,col1,col2,col3,col4,col5,col6,col7,col8
0,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449
1,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449
2,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449
3,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449
4,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449
5,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449
6,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449
7,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449,3.122449


Now that we have an equally sized imputed training set and raw average dataframes we can calculate the RMSE below:

In [112]:
training_rmse = sqrt(mean_squared_error(train_imputed, train_raw_avg_df))
print(f'The traning set RMSE is: {"%.4f" % training_rmse}')

The traning set RMSE is: 1.3066


*----------------*

Now let's go to work on the test set...

In [113]:
test

,Rock,Jazz,Pop,HipHop,Classical,Blues,Country,Folk
Users,,,,,,,,
user10,NaN,5.0,4.0,1.0,5.0,3.0,1.0,1.0
user9,2.0,2.0,5.0,2.0,3.0,1.0,2.0,4.0


In [114]:
# my method for finding the raw avgs. Sum the rows, then sum the column
test_row_sum = test.sum(axis=1)
df_test_sum = test_row_sum.sum()
test_raw_avg = df_test_sum / test.count().sum()
print(f'The raw average for the test set is: {"%.4f" % test_raw_avg}')

The raw average for the test set is: 2.7333


As we did with the training set, we impute the raw average to replace the NaNs in the test set.

In [115]:
test_imputed = test.fillna(test_raw_avg)
test_imputed

,Rock,Jazz,Pop,HipHop,Classical,Blues,Country,Folk
Users,,,,,,,,
user10,2.733333,5.0,4.0,1.0,5.0,3.0,1.0,1.0
user9,2.000000,2.0,5.0,2.0,3.0,1.0,2.0,4.0


Just as with the training set, we create a new dataframe full of raw averages for the test set in order to calcluate the test set RMSE.

In [116]:
test_avg = {
    "col1": [test_raw_avg for i in range(2)],
    "col2": [test_raw_avg for i in range(2)],
    "col3": [test_raw_avg for i in range(2)],
    "col4": [test_raw_avg for i in range(2)],
    "col5": [test_raw_avg for i in range(2)],
    "col6": [test_raw_avg for i in range(2)],
    "col7": [test_raw_avg for i in range(2)],
    "col8": [test_raw_avg for i in range(2)],
}

test_raw_avg_df = pd.DataFrame(test_avg)
test_raw_avg_df

,col1,col2,col3,col4,col5,col6,col7,col8
0,2.733333,2.733333,2.733333,2.733333,2.733333,2.733333,2.733333,2.733333
1,2.733333,2.733333,2.733333,2.733333,2.733333,2.733333,2.733333,2.733333


Lastly the RMSE is calculated when we subtract the equally sized imputed dataframe from the raw average dataframe with the output below.

In [117]:
test_rmse = sqrt(mean_squared_error(test_imputed, test_raw_avg_df))
print(f'The test set RMSE is: {"%.4f" % test_rmse}')

The test set RMSE is: 1.4347


*----------------*
The next part is to calculate the bias for each user and each item, and calculate the baseline predictors for every user-item combination.

In [118]:
train

,Rock,Jazz,Pop,HipHop,Classical,Blues,Country,Folk
Users,,,,,,,,
user1,1.0,1.0,4.0,1.0,4.0,NaN,NaN,3.0
user8,NaN,NaN,NaN,3.0,2.0,5.0,2.0,NaN
user5,5.0,2.0,4.0,NaN,2.0,1.0,3.0,5.0
user2,2.0,1.0,2.0,1.0,NaN,1.0,4.0,5.0
user4,4.0,5.0,2.0,NaN,4.0,NaN,5.0,5.0
user3,3.0,NaN,NaN,2.0,5.0,5.0,2.0,4.0
user6,5.0,3.0,5.0,4.0,4.0,2.0,NaN,1.0
user7,5.0,2.0,1.0,5.0,2.0,4.0,NaN,NaN


After viewing the training set we grab the sums of the rows and columns to make sure our math is correct before we calculate row and column bias values. 

In [119]:
print(train.sum(axis=0), train.count(axis=0))

Rock         25.0
Jazz         14.0
Pop          18.0
HipHop       16.0
Classical    23.0
Blues        18.0
Country      16.0
Folk         23.0
dtype: float64 Rock         7
Jazz         6
Pop          6
HipHop       6
Classical    7
Blues        6
Country      5
Folk         6
dtype: int64


In [120]:
train_col_bias = (train.sum(axis=0) / train.count(axis=0)) - train_raw_avg
train_col_bias

Rock         0.448980
Jazz        -0.789116
Pop         -0.122449
HipHop      -0.455782
Classical    0.163265
Blues       -0.122449
Country      0.077551
Folk         0.710884
dtype: float64

In [121]:
train_row_bias = (train.sum(axis=1) / train.count(axis=1)) - test_raw_avg
train_row_bias

Users
user1   -0.400000
user8    0.266667
user5    0.409524
user2   -0.447619
user4    1.433333
user3    0.766667
user6    0.695238
user7    0.433333
dtype: float64

The next step is to create a new matrix that gives us baseline predictors for every user-item combination and afterwards calculate the RMSE.

In [122]:
baseline_train = {
    "col1":[(train_col_bias[0] + i + train_raw_avg) for i in train_row_bias],
    "col2":[(train_col_bias[1] + i + train_raw_avg) for i in train_row_bias],
    "col3":[(train_col_bias[2] + i + train_raw_avg) for i in train_row_bias],
    "col4":[(train_col_bias[3] + i + train_raw_avg) for i in train_row_bias],
    "col5":[(train_col_bias[4] + i + train_raw_avg) for i in train_row_bias],
    "col6":[(train_col_bias[5] + i + train_raw_avg) for i in train_row_bias],
    "col7":[(train_col_bias[6] + i + train_raw_avg) for i in train_row_bias],
    "col8":[(train_col_bias[7] + i + train_raw_avg) for i in train_row_bias],
}

baseline_train_df = pd.DataFrame(baseline_train)
baseline_train_df

,col1,col2,col3,col4,col5,col6,col7,col8
0,3.171429,1.933333,2.600000,2.266667,2.885714,2.600000,2.800000,3.433333
1,3.838095,2.600000,3.266667,2.933333,3.552381,3.266667,3.466667,4.100000
2,3.980952,2.742857,3.409524,3.076190,3.695238,3.409524,3.609524,4.242857
3,3.123810,1.885714,2.552381,2.219048,2.838095,2.552381,2.752381,3.385714
4,5.004762,3.766667,4.433333,4.100000,4.719048,4.433333,4.633333,5.266667
5,4.338095,3.100000,3.766667,3.433333,4.052381,3.766667,3.966667,4.600000
6,4.266667,3.028571,3.695238,3.361905,3.980952,3.695238,3.895238,4.528571
7,4.004762,2.766667,3.433333,3.100000,3.719048,3.433333,3.633333,4.266667


In [123]:
baseline_train_rmse = sqrt(mean_squared_error(baseline_train_df, train_raw_avg_df))
print(f'The RMSE for the baseline predictors for the training data is: {"%.4f" % baseline_train_rmse}')

The RMSE for the baseline predictors for the training data is: 0.8248


MAYBE DELETE: And now we perform the same operation on the test data: Calculate the row and column bias, their baseline predictors, and find the RMSE.  

In [124]:
#test_col_bias = (test.sum(axis=0) / test.count(axis=0)) - test_raw_avg
#test_col_bias

In [125]:
#test_row_bias = (test.sum(axis=1) / test.count(axis=1) - test_raw_avg)
#test_row_bias

In [126]:
#baseline_test = {
    "col1":[(test_col_bias[0] + i + test_raw_avg) for i in test_row_bias],
    "col2":[(test_col_bias[1] + i + test_raw_avg) for i in test_row_bias],
    "col3":[(test_col_bias[2] + i + test_raw_avg) for i in test_row_bias],
    "col4":[(test_col_bias[3] + i + test_raw_avg) for i in test_row_bias],
    "col5":[(test_col_bias[4] + i + test_raw_avg) for i in test_row_bias],
    "col6":[(test_col_bias[5] + i + test_raw_avg) for i in test_row_bias],
    "col7":[(test_col_bias[6] + i + test_raw_avg) for i in test_row_bias],
    "col8":[(test_col_bias[7] + i + test_raw_avg) for i in test_row_bias],
}


#baseline_test_df = pd.DataFrame(baseline_test)
#baseline_test_df

IndentationError: unexpected indent (<ipython-input-126-44465a2b6fbb>, line 2)

In [127]:
#baseline_test_rmse = sqrt(mean_squared_error(baseline_test_df, test_raw_avg_df))
#print(f'The RMSE for the baseline predictors for the test data is: {"%.4f" % baseline_test_rmse}')

Lastly, we compare the original RMSE vs. the RMSE of the baseline predictors for the training and test data.

In [128]:
print(f'Original training set RMSE: {"%.4f" % training_rmse}',
     f'\nBaseline predictors training set RMSE: {"%.4f" % baseline_train_rmse}')

Original training set RMSE: 1.3066 
Baseline predictors training set RMSE: 0.8248


In [129]:
print(f'Original test set RMSE: {"%.4f" % test_rmse}',
     f'\nBaseline predictors test set RMSE: {"%.4f" % baseline_test_rmse}')

Original test set RMSE: 1.4347 
Baseline predictors test set RMSE: 1.1912


In [130]:
pct_improve_testdata = (1 - baseline_test_rmse / test_rmse) * 100
print(f'There was a {"%.1f" % pct_improve_testdata}% improvement with baseline predictors over the original test data RMSE')

There was a 17.0% improvement with baseline predictors over the original test data RMSE
